In [2]:
import pandas as pd

In [3]:
df_treino = pd.read_csv('./source_files/files/treino/treino_parte1.csv', delimiter=',')
df_treino.drop(columns=['timestampHistory_new'], inplace=True)

In [ ]:
len(df_treino)

100000

In [ ]:
batch_size = 1000
num_batches = len(df_treino) // batch_size + (1 if len(df_treino) % batch_size else 0)

In [ ]:
num_batches

100

In [81]:
user01 = df_treino.at[0, 'userId']
df_treino01 = df_treino[df_treino['userId'] == user01]

history
timestampHistory
numberOfClicksHistory
timeOnPageHistory
scrollPercentageHistory
pageVisitsCountHistory

In [ ]:
colums_explode = ["history", "timestampHistory", "numberOfClicksHistory", "timeOnPageHistory", "scrollPercentageHistory","pageVisitsCountHistory"]
df_treino01[colums_explode] = df_treino01[colums_explode].apply(lambda col: col.str.split(','))

C:\Users\felip\AppData\Local\Temp\ipykernel_17728\3381283319.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_treino01[colums_explode] = df_treino01[colums_explode].apply(lambda col: col.str.split(','))


In [60]:
df_treino01 = df_treino01.explode(colums_explode)

In [83]:
df_treino01

,userId,userType,historySize,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory
0,f98d1132f60d46883ce49583257104d15ce723b3bbda21...,Non-Logged,3,"[c8aab885-433d-4e46-8066-479f40ba7fb2, 68d203...","[1657146417045, 1657146605778, 1657146698738]","[76, 38, 41]","[20380, 21184, 35438]","[50.3, 18.18, 16.46]","[2, 1, 1]"


In [84]:
for _, row in df_treino01.iterrows():
    print(row)

userId                     f98d1132f60d46883ce49583257104d15ce723b3bbda21...
userType                                                          Non-Logged
historySize                                                                3
history                    [c8aab885-433d-4e46-8066-479f40ba7fb2,  68d203...
timestampHistory             [1657146417045,  1657146605778,  1657146698738]
numberOfClicksHistory                                         [76,  38,  41]
timeOnPageHistory                                    [20380,  21184,  35438]
scrollPercentageHistory                               [50.3,  18.18,  16.46]
pageVisitsCountHistory                                           [2,  1,  1]
Name: 0, dtype: object


In [ ]:
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.orm import sessionmaker, declarative_base

Base = declarative_base()

class TableTrain(Base):
    __tablename__ = 'TABLE_TRAIN'
    id = Column(Integer, primary_key=True)
    USERID = Column(String)
    USER_TYPE = Column(String)
    HISTORY_SIZE = Column(Integer)
    HISTORY = Column(String)
    TIME_STAMP_HISTORY = Column(Integer)
    NUMBER_OF_CLICKS_HISTORY = Column(Integer)
    TIME_ON_PAGE_HISTORY = Column(Integer)
    SCROLL_PERCENTAGE_HISTORY = Column(Integer)
    PAGE_VISITS_COUNT_HISTORY = Column(Integer)

db_path = "/globo.db"
engine = create_engine(f"sqlite://{db_path}", echo=True)

Base.metadata.drop_all(engine)
Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)
session = Session()

2025-02-12 22:17:04,593 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-02-12 22:17:04,593 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("TABLE_TRAIN")
2025-02-12 22:17:04,594 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-02-12 22:17:04,594 INFO sqlalchemy.engine.Engine COMMIT


In [67]:
for _, row in df_treino01.iterrows():
    new_row = TableTrain(
        USERID = row['userId'],
        USER_TYPE = row['userType'],
        HISTORY_SIZE = row['historySize'],
        HISTORY = row['history'],
        TIME_STAMP_HISTORY = row['timestampHistory'],
        NUMBER_OF_CLICKS_HISTORY = row['numberOfClicksHistory'],
        TIME_ON_PAGE_HISTORY = row['timeOnPageHistory'],
        SCROLL_PERCENTAGE_HISTORY = row['scrollPercentageHistory'],
        PAGE_VISITS_COUNT_HISTORY = row['pageVisitsCountHistory']
    )
    session.add(new_row)

session.commit()

2025-02-12 22:10:21,267 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-02-12 22:10:21,269 INFO sqlalchemy.engine.Engine INSERT INTO "TABLE_TRAIN" ("USERID", "USER_TYPE", "HISTORY_SIZE", "HISTORY", "TIME_STAMP_HISTORY", "NUMBER_OF_CLICKS_HISTORY", "TIME_ON_PAGE_HISTORY", "SCROLL_PERCENTAGE_HISTORY", "PAGE_VISITS_COUNT_HISTORY") VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?) RETURNING id
2025-02-12 22:10:21,270 INFO sqlalchemy.engine.Engine [generated in 0.00012s (insertmanyvalues) 1/3 (ordered; batch not supported)] ('f98d1132f60d46883ce49583257104d15ce723b3bbda2147c1e31ac76f0bf069', 'Non-Logged', 3, 'c8aab885-433d-4e46-8066-479f40ba7fb2', '1657146417045', '76', '20380', '50.3', '2')
2025-02-12 22:10:21,271 INFO sqlalchemy.engine.Engine INSERT INTO "TABLE_TRAIN" ("USERID", "USER_TYPE", "HISTORY_SIZE", "HISTORY", "TIME_STAMP_HISTORY", "NUMBER_OF_CLICKS_HISTORY", "TIME_ON_PAGE_HISTORY", "SCROLL_PERCENTAGE_HISTORY", "PAGE_VISITS_COUNT_HISTORY") VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?) RETURNING id
2025

In [68]:
from sqlalchemy import select

In [ ]:
stmt = select(TableTrain)
with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(row)

2025-02-12 22:20:37,523 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-02-12 22:20:37,523 INFO sqlalchemy.engine.Engine SELECT "TABLE_TRAIN".id, "TABLE_TRAIN"."USERID", "TABLE_TRAIN"."USER_TYPE", "TABLE_TRAIN"."HISTORY_SIZE", "TABLE_TRAIN"."HISTORY", "TABLE_TRAIN"."TIME_STAMP_HISTORY", "TABLE_TRAIN"."NUMBER_OF_CLICKS_HISTORY", "TABLE_TRAIN"."TIME_ON_PAGE_HISTORY", "TABLE_TRAIN"."SCROLL_PERCENTAGE_HISTORY", "TABLE_TRAIN"."PAGE_VISITS_COUNT_HISTORY" 
FROM "TABLE_TRAIN"
2025-02-12 22:20:37,524 INFO sqlalchemy.engine.Engine [cached since 152.8s ago] ()
(1, 'f98d1132f60d46883ce49583257104d15ce723b3bbda2147c1e31ac76f0bf069', 'Non-Logged', 3, 'c8aab885-433d-4e46-8066-479f40ba7fb2', 1657146417045, 76, 20380, 50.3, 2)
(2, 'f98d1132f60d46883ce49583257104d15ce723b3bbda2147c1e31ac76f0bf069', 'Non-Logged', 3, ' 68d2039c-c9aa-456c-ac33-9b2e8677fba7', 1657146605778, 38, 21184, 18.18, 1)
(3, 'f98d1132f60d46883ce49583257104d15ce723b3bbda2147c1e31ac76f0bf069', 'Non-Logged', 3, ' 13e423ce-1d69-4c78-